In [ ]:
!unzip -FF merged_dataset.zip


Streaming output truncated to the last 5000 lines.
  inflating: content/merged_dataset/train/labels/0_IMG_20211006_143018_jpg.rf.4be0f292abd5d894eae58a6f06ac67b9.txt  
  inflating: content/merged_dataset/train/labels/3_image_4_jpg.rf.dfb13ed765f36d4cd5bbb700d7cfb747.txt  
  inflating: content/merged_dataset/train/labels/8_images-29-_jpeg.rf.30834f769051edf9c9a4a6a23d745bc5.txt  
  inflating: content/merged_dataset/train/labels/6_5_png_jpg.rf.38c0b3e7fc9a9d32fb9a91e2ab6da389.txt  
  inflating: content/merged_dataset/train/labels/1_18_png.rf.7db1f63e9d66b21c604879f2b44604e4.txt  
  inflating: content/merged_dataset/train/labels/2_2_png_jpg.rf.222910393de912b1b772e5268d06acc4.txt  
  inflating: content/merged_dataset/train/labels/1_19_png.rf.cc72311e035e9fe581d03d57fa58929c.txt  
  inflating: content/merged_dataset/train/labels/8_IC_0242_jpg.rf.9369ed47bdd34f261b1f0b27c9974b16.txt  
  inflating: content/merged_dataset/train/labels/8_tranzistor_679_jpg.rf.068f71552d6df915c6add79e29841f95.t

In [ ]:
!mv /content/content/merged_dataset /content/merged_dataset

In [ ]:
!rm /content/merged_dataset.zip

In [ ]:
import yaml
from pathlib import Path

root = Path("/content/merged_dataset")

# Load classes
with open(root / "data.yaml", "r") as f:
    data = yaml.safe_load(f)
names = data["names"]
nc = len(names)

# Storage for counts
counts = {i: 0 for i in range(nc)}

splits = ["train", "valid", "test"]

for split in splits:
    labels_dir = root / split / "labels"
    if not labels_dir.exists():
        continue

    for txt in labels_dir.glob("*.txt"):
        found_classes = set()
        for line in txt.read_text().splitlines():
            if not line.strip():
                continue
            cls = int(line.split()[0])
            found_classes.add(cls)

        # Count each class once per image
        for c in found_classes:
            counts[c] += 1

# Print results
print("\nClass-wise image counts:")
for i, name in enumerate(names):
    print(f"{i:<3} {name:<25} {counts[i]}")


Class-wise image counts:
0   ceramic_capacitor         323
1   potentiometer             229
2   resistor                  1646
3   diode                     406
4   electrolytic_capacitor    458
5   polyester_capacitor       91
6   transistor                1265
7   ic                        751
8   led                       1150


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 99.6 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")  # start with s

model.train(
    data="/content/merged_dataset/data.yaml",
    epochs=100,
    imgsz=640,
    batch=64,        # lower to 32 if you ever hit CUDA OOM
    device=0,        # A100
    workers=8,
    patience=25,
    cos_lr=True,
    optimizer="SGD", # fine; you can also omit this and let YOLO choose
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA H100 80GB HBM3, 81224MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/merged_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d9a61f2f830>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [ ]:
!ls /content/runs/detect/train/weights

best.pt  last.pt


In [ ]:
from google.colab import files
files.download("/content/runs/detect/train/weights/best.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>